In [ ]:

from collections import Counter
import re
import string
from random import shuffle

from subprocess import call
from IPython.display import Image

!pip install xlrd
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import spacy
from scipy import stats

!pip install textblob
from textblob import TextBlob

!pip install textstat
import textstat

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("ggplot")

from scipy import stats

!python -m spacy download en
import spacy

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
!pip install yellowbrick
from yellowbrick.classifier import ClassificationReport

!pip install empath
from empath import Empath

from sklearn.model_selection import RandomizedSearchCV
!apt-get install graphviz -y
from sklearn.tree import export_graphviz

In [ ]:
#Reading in Non_Russians data set
Non_Russian=pd.read_csv("csv-zusammenfuehren.de_3mjg6fs7.csv", error_bad_lines=False)

In [ ]:
#Reading in comments by Russian Trolls
Russians=pd.read_csv("comments.csv")

In [ ]:
Non_Russian=Non_Russian[["body",'controversiality','created_utc',"subreddit","score"]]

In [ ]:
Russians=Russians[["body","controversiality","created_utc","subreddit.display_name_prefixed","score"]]

In [ ]:
Non_Russian.head()

# Subreddit EDA

In [ ]:
Russians.describe()

In [ ]:
#Trolls Top Subreddits
Russians.head()
ax=Russians["subreddit.display_name_prefixed"].value_counts()[:10].plot(kind="bar",title="Top Subreddits for Russian Trolls")
ax.set_xlabel("SubReddit Name")
ax.set_ylabel("Number of comments from that subreddit")

In [ ]:
#Top Non_Russian Subreddits
Non_Russian.head()
ax=Non_Russian["subreddit"].value_counts()[:10].plot(kind="bar",title="Top Subreddits for Regular Users")
ax.set_xlabel("SubReddit Name")
ax.set_ylabel("Number of comments from that subreddit")

In [ ]:
#How many non_russian comments are there
len(Non_Russian.drop_duplicates())

In [ ]:
#Sample the same number of non-russian comments as russian comments
Non_Russians_Sample=Non_Russian.sample(n=len(Russians),random_state=1)

In [ ]:
#Top SubReddits from random sample
Non_Russians_Sample["subreddit"].value_counts()[:10].plot(kind="bar")

In [ ]:
#How many top subreddits do both datasets contain
Non_Russians_Sample["subreddit"].value_counts()[:20].index.isin(Non_Russian["subreddit"].value_counts()[:20].index)

In [ ]:
#Listing twenty most common subreddits
Non_Russian["subreddit"].value_counts()[:20].index

# EDA UpVotes

In [ ]:
#Most common amount of upvotes received for Russian Trolls
print(Russians.score.value_counts()[:10])



In [ ]:
#Most common amount of upvotes received for regular users
print(Non_Russians_Sample.score.value_counts()[:10])

In [ ]:
#What is the mean amount of upvotes received?
print("The mean number of upvotes received for regular users was was {}".format
     (np.mean(Non_Russians_Sample.score)))

print("The mean number of upvotes received for Trolls was {}".format
     ((np.mean(Russians.score))))


In [ ]:
#100th percentile of upvotes for regular users
print(np.percentile(Non_Russians_Sample.score.dropna(),100))

In [ ]:
#100th percentile of upvotes for trolls
print(np.percentile(Russians.score.dropna(),100))

In [ ]:
#Binning comments by number of upvotes for Non_Russians
bins=[-10,0,10,100,500,1000]#Bins
digitized=np.digitize(Non_Russians_Sample.score, bins)
bin_means=[Non_Russians_Sample.score[digitized == i].mean() for i in range(0, len(bins))]
print(bin_means) #mean upvotes in every binned group

In [ ]:
#Visualizing the distribution of bins
ax=sns.countplot(digitized)
ax.set(xlabel='bin', ylabel='Number of comments',title="Distributions of Upvotes for Non_Russian Comments")

In [ ]:
#Binning comments by number of upvotes for Russians
bins = [-10,0,10,100,500,1000]
digitized = np.digitize(Russians.score, bins)
bin_means = [Russians.score[digitized == i].mean() for i in range(0, len(bins))]


In [ ]:
ax=sns.countplot(digitized)
ax.set(xlabel='bin', ylabel='Number of comments',title="Distributions of Upvotes for Russian Comments")

# EDA Time

In [ ]:
#converting utc to datetime
Russians["date"] = pd.to_datetime(Russians['created_utc'],unit='s')
Non_Russians_Sample["date"] = pd.to_datetime(Non_Russians_Sample['created_utc'],unit='s')

In [ ]:
#Converting utic to datetime
Russians["Time"]=Russians["date"].dt.hour.dropna()
Non_Russians_Sample["Time"]=Non_Russians_Sample["date"].dt.hour.dropna()


In [ ]:
ax=sns.countplot(Non_Russians_Sample.Time)
ax.set(title="Distribution of times that comments were posted",xlabel="time",ylabel="Count")

In [ ]:
ax=sns.countplot(Russians.Time)
ax.set(title="Distribution of times that comments were posted for Russians",xlabel="time",ylabel="Count")

In [ ]:
#Checking to see if Distributions of Time Posted is statistically significant
stats.ks_2samp(Russians.Time, Non_Russians_Sample.Time)

In [ ]:
#Dates with the most comments by Russian Trolls
Russians["date"].dt.date.value_counts()[:50]

In [ ]:
#Dates with the most comments by regular Reddit Users
Non_Russians_Sample["date"].dt.date.value_counts()[:20]

Extracting Named Entities

In [ ]:
!pip install msgpack-python==0.5.6


In [ ]:
#Loading 

nlp = spacy.load('en')

In [ ]:
import msgpack

In [ ]:

#cleaning up both datasets
Russians.body=Russians.body.str.replace("(\r)|(\n)","")
Non_Russians_Sample.body=Non_Russians_Sample.body.str.replace("(\r)|(\n)","")

In [ ]:
def named_entities(x):
    """This Function Extracts Named Entities"""
    doc=nlp(x)
    named_=[]
    for doc in doc.ents:
        if doc == None:
            return named_
        else:
            named_.append(doc)
    return named_

In [ ]:
#Applying function
Non_Russians_Sample["Named_entities"]=Non_Russians_Sample.body.apply(func=lambda x: named_entities(x))

In [ ]:
#Applying function
Russians["Named_entities"]=Russians.body.apply(func=lambda x: named_entities(x))

In [ ]:
def get_polarity(x):
  """This Function calculates polarity for every comment."""
  b = TextBlob(x)
  return b.sentiment.polarity

In [ ]:
#Getting the polarity of Non_Russian comments
Non_Russians_Sample["Polarity"]=Non_Russians_Sample.body.apply(func=lambda x: get_polarity(x))
np.mean(Non_Russians_Sample["Polarity"])

In [ ]:
#Getting the Polarity of Russian comments
Russians["Polarity"]=Russians.body.apply(func=lambda x: get_polarity(x))
np.mean(Russians["Polarity"])

Flesch-Kinkaid

In [ ]:
#Average Flesch Kincaid For Trolls
Russians["Flesch_Kincaid"]=Russians.body.apply(func=lambda x: textstat.flesch_kincaid_grade(x))
np.mean(Russians["Flesch_Kincaid"])

In [ ]:
#Average Flesch Kincaid for Regular Users
Non_Russians_Sample["Flesch_Kincaid"]=Non_Russians_Sample.body.apply(func=lambda x: textstat.flesch_kincaid_grade(x))
np.mean(Non_Russians_Sample["Flesch_Kincaid"])

# Controversiality

In [ ]:
#Number of Russian Comments that were marked as controversial
Russians.controversiality.value_counts()

In [ ]:
#Number of Non-Russian Samples marked as controversial
Non_Russians_Sample.controversiality.value_counts()

# Lexical Categories

In [ ]:
#initializing Lexicon
lexicon = Empath()

In [ ]:
#Creating 5 lexical categories related to various subjects that I hypothesize Russian Trolls would be talking about
lexicon.create_category("russia",["russia","putin","moscow","kremlin","KGB","ukraine","russian"],model="reddit")
lexicon.create_category("social_media",["fake_news","twitter_bot","social_media"],model="reddit")
lexicon.create_category("trump",["donald_trump","POTUS","michael_flynn","russian_collusion","mueller_investigation","putin"],model="reddit")
lexicon.create_category("FBI",["FBI","investigation","robert","comey","flynn"],model="reddit")
lexicon.create_category("hilary",["DNC","emails","bernie"],model="reddit")

In [ ]:
def Lexical_Score(x):
    """This Function looks at every comment and gives it a lexical score depending 
    on the context of the sentence"""
    c=0
    analyze=lexicon.analyze(x,categories=['russia,social_media',"trump","FBI","hilary"],normalize=True)
    attrs=[w for k,w in analyze.items() if w >0]
    if len(attrs)>0:
        for attr in attrs:
            c+=attr
    return c

In [ ]:
Russians["Lexical_Score"]=Russians.body.apply(func=lambda x:Lexical_Score(x))

In [ ]:
Non_Russians_Sample["Lexical_Score"]=Non_Russians_Sample.body.apply(func=lambda x:Lexical_Score(x))

# Adding Other word based features

In [ ]:
#word count
Russians['word_count']=Russians['body'].apply(lambda x: len(x.split()))
Non_Russians_Sample['word_count']=Non_Russians_Sample['body'].apply(lambda x: len(x.split()))

In [ ]:
#Punctuation Count
Russians['punctuation_count']=Russians['body'].apply(lambda x: len("".join(punc for punc in x if punc in string.punctuation))) 
Non_Russians_Sample['punctuation_count']=Non_Russians_Sample['body'].apply(lambda x: len("".join(punc for punc in x if punc in string.punctuation))) 

In [ ]:
#Number of Capital Words
Russians['upper_case_word_count'] = Russians.body.apply(lambda x: len([word for word in x.split() if word.isupper()]))
Non_Russians_Sample['upper_case_word_count'] = Non_Russians_Sample.body.apply(lambda x: len([word for word in x.split() if word.isupper()]))

# Checking if Comments appear on significant dates

In [ ]:
#Reading in the russia timeline sheet
Russia_Timeline=pd.read_excel("Russia_Timeline.xlsx",header=1)

In [ ]:
#Cleaning up excel spreadsheet
years=[2014,2015,2016,2017]
Filtered_Timeline=Russia_Timeline[Russia_Timeline.DATE.isin(years)]
twenty_eight=Russia_Timeline[Russia_Timeline.DATE.isin([2018])]

In [ ]:
#Extracting day
Russians["Day"]=Russians.date.dt.date
Non_Russians_Sample["Day"]=Non_Russians_Sample.date.dt.date

In [ ]:
#Combining dates in excel spreadsheet
combined_dates=Filtered_Timeline["DATE"].astype(str)+" "+ Filtered_Timeline['Unnamed: 1'].astype(str)

In [ ]:
#Cleaning up dates
twenty_eight['Unnamed: 1']=twenty_eight['Unnamed: 1'].astype(str)
twenty_eight=twenty_eight['Unnamed: 1'].str.replace("(00:00:00)","")

In [ ]:
#Recombining dates
other=pd.to_datetime(combined_dates[combined_dates.str.contains("\w+ \w+ \d+")],errors="coerce")
all_together=other.append(twenty_eight)

In [ ]:
#dropping N/A rows
all_together.dropna(inplace=True)

In [ ]:
relevant_dates=pd.to_datetime(all_together.astype(str).str.replace("(00:00:00)",""),errors="coerce")

In [ ]:
def date_it(x):
    """ If a comment takes place on a relevant date return 1
     Otherwise return 0"""
    if x in list(relevant_dates):
        return 1
    else:
        return 0

In [ ]:
#Applying function
relevant_dates=relevant_dates.astype(str)
Russians["Day"]=Russians.Day.astype(str) + " " + "00:00:00"
Russians["Relevant_Date"]=Russians.Day.apply(lambda x: date_it(x))
Non_Russians_Sample["Day"]=Non_Russians_Sample.Day.astype(str) + " " + "00:00:00"
Non_Russians_Sample["Relevant_Date"]=Non_Russians_Sample.Day.apply(lambda x: date_it(x))

In [ ]:
#Filtering dfs for columns I think are meaningful
Non_Russians_Sample=Non_Russians_Sample[["Time","Named_entities","Polarity",
                     "Flesch_Kincaid","Lexical_Score","word_count","punctuation_count",
                     "upper_case_word_count","Relevant_Date"]]
Russians=Russians[["Time","Named_entities","Polarity",
                     "Flesch_Kincaid","Lexical_Score","word_count","punctuation_count",
                     "upper_case_word_count","Relevant_Date"]]


In [ ]:
#Adding Labels
Non_Russians_Sample["Label"]=["0" for x in range(len(Non_Russians_Sample))]
Russians["Label"]=["1" for x in range(len(Non_Russians_Sample))]

In [ ]:
#Appending both dataframes
total=Non_Russians_Sample.append(Russians)

In [ ]:
#Counting total number of named entities for every comment
total["Named_entities"]=total.Named_entities.apply(lambda x:len(x))

# Baseline Test

In [ ]:
#Adding a to signify russian or non-russian
Russians["Troll_Status"]=["Russian" for x in range(len(Russians))]
Non_Russians_Sample["Troll_Status"]=["Non-Russian" for x in range(len(Russians))]
appended=np.array(Non_Russians_Sample.append(Russians)["Troll_Status"])

In [ ]:
#Creating a list with random values of russians and non-russians
actual=list(pd.Series(appended))
troll=["Russian" for x in range(len(Russians))]
not_troll=["Non-Russian" for x in range(len(Russians))]
combined=troll+not_troll

In [ ]:

shuffle(combined)
shuffle(appended)

In [ ]:
#Calculating Accuracy
x=0
for label in range(len(actual)):
    if actual[label]==combined[label]:
        x+=1
print(x/len(combined)*100)

In [ ]:
#Random Predictions
actual_converted=[]
test=[]
for label in actual:
  if label=="Non-Russian":
    actual_converted.append(0)
  else:
    actual_converted.append(1)
for label in combined:
  if label=="Non-Russian":
    test.append(0)

  else:
    test.append(1)

In [ ]:
#Calculating Recall
metrics.recall_score(actual_converted,test)

In [ ]:
#Calculating AUC
metrics.roc_auc_score(actual_converted,test)

# Using Machine Learning for Predictions

In [ ]:
#Creating a new index
total.index=list(range(len(total)))


In [ ]:
#Randomly Shuffling rows
shuffled_rows=total.reindex(np.random.permutation(total.index))

In [ ]:
#Dropping rows with N/A
shuffled_rows.dropna(inplace=True)

In [ ]:

X=shuffled_rows.iloc[:,:-1]#Independent Variables
y=shuffled_rows.iloc[:,-1]#Value I want to predict

In [ ]:
f, ax = plt.subplots(figsize=(10, 6))
corr = X.corr()
hm = sns.heatmap(round(corr,2), annot=True, ax=ax, cmap="coolwarm",fmt='.2f',
                 linewidths=.05)
f.subplots_adjust(top=0.93)
t= f.suptitle('Model Attributes Correlation Heatmap', fontsize=14)

In [ ]:
#Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)  

In [ ]:
#Standardizing Data
sc = StandardScaler()  
X_train = sc.fit_transform(X_train)  
X_test = sc.transform(X_test)  

In [ ]:
#Fitting a Random Forest Classifier
Classifier = RandomForestClassifier(n_estimators=20, random_state=0)  
Classifier.fit(X_train, y_train)  
y_pred = Classifier.predict(X_test) 

In [ ]:
y_pred=y_pred.astype(int)
y_test_int=y_test.astype(int)

In [ ]:
#Calculating Recall
metrics.recall_score(y_test_int,y_pred)

In [ ]:
#Calculating auc
metrics.roc_auc_score(y_test_int,y_pred)

In [ ]:
visualizer = ClassificationReport(RandomForestClassifier() ,cmap='BuPu',title="Normal Random Forest Parameters",classes=["Non-Troll","Troll"], support=True)
visualizer.fit(X_train, y_train)  # Fit the visualizer and the model
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
g = visualizer.poof()             # Draw/show/poof the data

In [ ]:
#Evaluation Metrics
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred)) 

# Using Random Search CV to find optimal Parameters Code Partially Taken From: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]


In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
#Initializing the classifier
rf=RandomForestClassifier()

In [ ]:
#Initally
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, random_state=42, n_jobs = -1,
                              scoring="accuracy")

In [ ]:
#Fitting Randomized Search
rf_random.fit(X_train, y_train)

In [ ]:
#Finding optimal parameters
grid_params=rf_random.best_params_
grid_params


In [ ]:
#initializing RF with parameters optimized
classifier_optimized = RandomForestClassifier(bootstrap=True,max_depth=10,max_features="sqrt",
                                            min_samples_leaf=1, min_samples_split=5, n_estimators=2000) 
classifier_optimized.fit(X_train,y_train)

In [ ]:
#Making Predictions
predictions=classifier_optimized.predict(X_test)

In [ ]:
fig,ax1 = plt.subplots()

visualizer = ClassificationReport(RandomForestClassifier(bootstrap=True,max_depth=10,max_features="sqrt",
                                            min_samples_leaf=1, min_samples_split=5, n_estimators=2000) ,title="Optimized Parameters",classes=["Non-Troll","Troll"], support=True)
visualizer.fit(X_train, y_train)  # Fit the visualizer and the model
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
g = visualizer.poof()             # Draw/show/poof the data

In [ ]:
#Evaluating new predictions
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions)) 

In [ ]:
y_test_int

In [ ]:
metrics.roc_auc_score(y_test_int,predictions.astype(int))

In [ ]:
#Printingo ut most import coefficients
for name, importance in zip(shuffled_rows.columns, classifier_optimized.feature_importances_):
    print(name, "=", importance)

In [ ]:
#Plotting out most important features
importances = classifier_optimized.feature_importances_
order_indices = np.argsort(importances)


plt.title('Feature Importances')
plt.xlabel("Coefficient Size")
plt.ylabel("Feature")
plt.barh(range(len(order_indices)), importances[order_indices],color=["b","r","g","y"], align='center')
plt.yticks(range(len(order_indices)), [shuffled_rows.columns[i] for i in order_indices])

In [ ]:
esti=classifier_optimized.estimators_[0]
esti

In [ ]:
#Creating a tree
export_graphviz(esti, out_file='tree.dot', feature_names=list(shuffled_rows)[:-1],
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png

call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display image

Image(filename = 'tree.png')
